In [1]:
import sys
import os
import json
import pandas as pd
sys.path.insert(0, '/home/igormago/projects/DeepFootball')

#### Defining variables

In [2]:
from config.configparser import config
experiments_path = config['path']['experiments']

In [18]:
class Model():
    
    def __init__(self, name, project, experiment, workflow):
        self.name = name
        self.project = project
        self.experiment = experiment
        self.workflow = workflow 
        
        self.results = self._get_results()
        self.label = self._get_label()
        
    
    def get_path(self):
        return os.path.join(experiments_path, project, experiment, workflow)
    
    def _get_results(self):
        
        acc_train = list()
        acc_test = list()
        rps_train = list()
        rps_test = list()

        for i in range(0,96):
            file_name = str(i) + '.json'
            file_path = os.path.join(self.get_path(), self.name, 'results', file_name)
            with open(file_path) as json_file:
                data = json.load(json_file)
                acc_train.append(data['acc_train'])
                acc_test.append(data['acc_test'])
                rps_train.append(data['rps_train'])
                rps_test.append(data['rps_test'])
                
        df = pd.DataFrame()
        df['acc_train'] = acc_train
        df['acc_test'] = acc_test
        df['rps_train'] = rps_train
        df['rps_test'] = rps_test
        df['minute'] = df.index
        
        return df
    
    def _get_label(self):
        return '_'.join([self.name, self.experiment, self.workflow])

In [33]:
models_list = []

project = 'realtime'

for model_name in ['LogisticRegression', 'GaussianNB']:
    for experiment in ['smbf_comp', 'smbf_comp_po', 'smbf_comp_io', 'smbf_comp_po_io', 'smbf_io']:
        workflow = 'StandardByMinuteForecasting'

        model = Model(model_name, project, experiment, workflow)
        models_list.append(model)

for model_name in ['XGBClassifier']:
    for experiment in ['smbf_comp', 'smbf_comp_po_io', 'smbf_io']:
        workflow = 'StandardByMinuteForecasting'

        model = Model(model_name, project, experiment, workflow)
        models_list.append(model)
        
        
for experiment in ['smbf_comp']:
    for model_name in ['Betfair']:
        workflow = 'MarketByMinuteForecasting'

        model = Model(model_name, project, experiment, workflow)
        models_list.append(model)

In [34]:
acc_train = pd.DataFrame()
acc_test = pd.DataFrame()
rps_train = pd.DataFrame()
rps_test = pd.DataFrame()

for m in models_list:
    
    acc_train[m.label] = m.results['acc_train']
    acc_test[m.label] = m.results['acc_test']
    rps_train[m.label] = m.results['rps_train']
    rps_test[m.label] = m.results['rps_test']


In [12]:
import plotly.io as pio
pio.renderers.default = 'iframe'

In [35]:
import plotly.express as px
import plotly.graph_objs as go

data = list()

for m in models_list:
    trace = go.Scatter(x=rps_test.index, y=rps_test[m.label], 
                                  mode='lines', name=m.label)

    data.append(trace)

layout = go.Layout(
    xaxis = go.layout.XAxis(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 5
    )
)

fig = go.Figure(data=data, layout=layout)

fig.show()